In [106]:
import psycopg2
import pandas as pd

In [56]:
#%run functions.ipynb

In [119]:
def do_query(query, print_out = False, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        cur.execute(query)
        
        for notice in con.notices:
            print(notice)

        if commit:
            con.commit()

        if print_out:
            column_names = [desc[0] for desc in cur.description]
            row_list = []
            row = cur.fetchone()
            while row is not None:
                row_list.append(row)
                #print(row)
                row = cur.fetchone()
            df = pd.DataFrame(row_list, columns=column_names)
            print(df)
        
        
        
        cur.close()
        con.close()
        print('\n')
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# FUNCTIONS

In [58]:
def function_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [118]:
def function_tester(function, *arguments_of_it):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()
        
        if len(arguments_of_it) == 1:
            cur.execute(function + "('" +str(arguments_of_it[0]) + "')")
        else:
            cur.execute(function + str(arguments_of_it))

        column_names = [desc[0] for desc in cur.description]
        row_list = []
        row = cur.fetchone()
        while row is not None:
            row_list.append(row)
            #print(row)
            row = cur.fetchone()
        df = pd.DataFrame(row_list, columns=column_names)
        print(df)
        
        for notice in con.notices:
            print(notice)

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [60]:
function_def("""
    CREATE OR REPLACE FUNCTION ai_med.get_sum(
        a NUMERIC, 
        b NUMERIC) 
    RETURNS NUMERIC AS $$
    BEGIN
        RETURN a + b;
    END; $$

    LANGUAGE plpgsql;
    """,
    commit = True)

In [116]:
function_tester("select ai_med.get_sum", 10, 20)

(Decimal('30'),)
  get_sum
0      30


In [62]:
function_def("""
CREATE OR REPLACE FUNCTION ai_med.test1 ()
RETURNS integer AS $total$
declare
	total integer;
BEGIN
   select count(*) into total from ai_med.pacjent;
   RETURN total;
END;
$total$ LANGUAGE plpgsql;
""",
commit = True)

In [63]:
function_tester("select ai_med.test1")

(50089,)


In [120]:
do_query("""
SELECT * FROM ai_med.pacjent WHERE id = 1;
""",
print_out=True)

   id  imie nazwisko data_urodzenia                       created_at
0   1  Robo     Bobo     1934-08-31 2022-11-20 15:03:59.028075+01:00




In [ ]:
# do_query("""
# drop function if exists ai_med.pacjenci_z_id(character, character);;
# """, 
# commit=True)

# function_def("""
# create or replace function ai_med.pacjenci_z_id(imieee char, nazwiskooo char)

# returns table(
#    data date,
#    data2 date,
#    dni int
# )

# language plpgsql
# as
# $$

# declare
#    ilosc_pacjentow integer;

# begin
   
#    return query
#    SELECT "Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma
#    ,"Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma
#    ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin"
#    FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
#    ON "Aktualnie przyjmowane leki".id = pacjent.id 
#    WHERE pacjent.imie = imieee or pacjent.nazwisko = nazwiskooo;

# end;
# $$;
#     """,
# commit = True)

In [113]:
do_query("""
drop function if exists ai_med.pacjenci_z_id(character, character);;
""", 
commit=True)

function_def("""
create or replace function ai_med.pacjenci_z_id(imieee char, nazwiskooo char)

returns table(
   data date,
   data2 date,
   dni int
)

language plpgsql
as
$$

declare
   ilosc_pacjentow integer;

begin
   
   return query
   SELECT "Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma
   ,"Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma
   ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin"
   FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
   ON "Aktualnie przyjmowane leki".id = pacjent.id 
   WHERE pacjent.imie = imieee or pacjent.nazwisko = nazwiskooo;

end;
$$;
    """,
commit = True)

NOTICE:  function ai_med.pacjenci_z_id(pg_catalog.bpchar,pg_catalog.bpchar) does not exist, skipping





In [111]:
do_query("""
drop function if exists ai_med.pacjenci_z_id(character, character);;
""", 
commit=True)

function_def("""
create or replace function ai_med.pacjenci_z_id(imieee char, nazwiskooo char, lek char)

returns table(
   data date,
   data2 date,
   dni int
)

language plpgsql
as
$$

declare
   ilosc_pacjentow integer;

begin
   
   return query
   if 
   SELECT "Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma
   ,"Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma
   ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin"
   FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
   ON "Aktualnie przyjmowane leki".id = pacjent.id 
   WHERE pacjent.imie = imieee or pacjent.nazwisko = nazwiskooo;

end;
$$;
    """,
commit = True)



syntax error at or near "if"
LINE 20:    if 
            ^



In [121]:
function_tester("select * FROM ai_med.pacjenci_z_id", "Rachelle", "Wheler")

         data       data2   dni
0  2018-05-07  2020-08-04   820
1  2018-07-30  2021-04-24   999
2  2019-04-02  2021-02-08   678
3  2018-10-13  2020-12-22   801
4  2019-03-23  2021-08-16   877
5  2018-05-25  2021-05-04  1075


In [67]:
function_def("""
    CREATE OR REPLACE FUNCTION ai_med.przyjmowanie_leku(
    
    ) 
    
    RETURNS numeric AS $$
    
    DECLARE total integer;

    BEGIN
        select count(*) into total from ai_med.pacjent;

        RETURN total;
    
    END; $$
    LANGUAGE plpgsql;
    """,
commit = True)

In [68]:
function_tester("select ai_med.przyjmowanie_leku")

(Decimal('50089'),)


In [69]:
function_def("""
    CREATE OR REPLACE FUNCTION ai_med.przyjmowanie_leku(
    
    ) 
    
    RETURNS numeric AS $$
    
    DECLARE total integer;

    BEGIN
        select count(*) into total from ai_med.pacjent;

        RETURN total;
    
    END; $$
    LANGUAGE plpgsql;
    """,
commit = True)

In [70]:
function_tester("select ai_med.przyjmowanie_leku")

(Decimal('50089'),)


# TRIGGERS

In [71]:
def trigger_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [72]:
trigger_def("""
CREATE TRIGGER pacjent_insert_timestamp 
    AFTER
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.pacjent_insert_timestamp_func();
""",
commit=True)

trigger "pacjent_insert_timestamp" for relation "pacjent" already exists



In [73]:
function_def("""
CREATE OR REPLACE FUNCTION ai_med.pacjent_insert_timestamp_func() RETURNS TRIGGER AS $pacjent_table$
   BEGIN
      INSERT INTO ai_med.pacjent_insert_timestamp_table(id, ostatni_insert_danych) VALUES (NEW.id ,date_trunc('day', current_timestamp));
      RETURN NEW;
   END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)

In [74]:
do_query("""
CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
(
    id serial NOT NULL,
    ostatni_insert_danych date NOT NULL,
    ostatnie_powiadomienie_dla_MLE date
);
""", 
commit=True,
print_out=True)

NOTICE:  relation "pacjent_insert_timestamp_table" already exists, skipping

no results to fetch


In [75]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Paul', 'California', '2022-11-23');
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent where imie ='Paul' and nazwisko = 'California';
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)


no results to fetch
(110732, 'Paul', 'California', datetime.date(2022, 11, 23), datetime.datetime(2022, 11, 23, 18, 55, 12, 167567, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)))
(110736, 'Paul', 'California', datetime.date(2022, 11, 23), datetime.datetime(2022, 11, 23, 21, 24, 50, 528890, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)))
(110737, 'Paul', 'California', datetime.date(2022, 11, 23), datetime.datetime(2022, 11, 23, 21, 25, 22, 232925, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)))
(110745, 'Paul', 'California', datetime.date(2022, 11, 23), datetime.datetime(2022, 11, 23, 22, 19, 13, 968228, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)))
(110747, 'Paul', 'California', datetime.date(2022, 11, 23), datetime.datetime(2022, 11, 23, 22, 21, 39, 707711, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)))
(110749, 'Paul', 'California', datetime.date(2022, 11, 23), datetime.datetime(2022, 11, 23, 22, 24, 6, 47

In [76]:
trigger_def("""
CREATE OR REPLACE TRIGGER message_to_ML_E
    BEFORE
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.message_to_ML_E_func();
""",
commit=True)

In [77]:
#TO W SUMIE POWINNO BYc NA KONIEC JAKIEJŚ TRANZAKCJI
#W SENSIE ŻEBY ZROBIĆ TRANZAKCJĘ, KTÓRA ZAJMIE SIĘ DAILY INSERTEM DO TABEL
function_def("""
CREATE OR REPLACE FUNCTION ai_med.message_to_ML_E_func() RETURNS TRIGGER AS $pacjent_table$
	declare
		liczba_wprowadzonych int;
	BEGIN
		SELECT count(*) into liczba_wprowadzonych from ai_med.pacjent_insert_timestamp_table 
		where ostatni_insert_danych = date_trunc('day', current_timestamp);
		IF liczba_wprowadzonych = 5
		THEN
		UPDATE ai_med.pacjent_insert_timestamp_table
		SET ostatnie_powiadomienie_dla_mle=date_trunc('minute', current_timestamp);
		raise notice 'Dzis zostalo wprowadzonych przynajmniej % nowych pacjentow, moze chcecie zrobic douczanie modelu?', 
		
		liczba_wprowadzonych + 1;
		
		END IF;
		RETURN NEW;
   	END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)



In [78]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Paul', 'California', '2002-11-23');
""", 
commit=True,
print_out=True)

do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)

no results to fetch
(110830, datetime.date(2022, 11, 24), None)
(110831, datetime.date(2022, 11, 24), None)




In [79]:
# do_query("""
# DROP TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
# """, 
# commit=True,
# print_out=True)

# do_query("""
# CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
# (
#     id serial NOT NULL,
#     ostatni_insert_danych date NOT NULL,
#     ostatnie_powiadomienie_dla_MLE date
# );
# """, 
# commit=True,
# print_out=True)

# PROCEDURES

In [80]:
def procedure_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# 1. Inserting data using a procedure GENEROWANIE danych za pomocą procedur?

In [81]:
procedure_def(
"""
create or replace procedure ai_med.select_all_pacjenty()
language plpgsql    
as $$
declare
    msg int;
begin

    select count(*) into msg from ai_med.pacjent;

    RAISE NOTICE 'W bazie jest juz: % pacjentow!!', msg ;

end;$$
""",
commit=True)

In [82]:
do_query("""
call ai_med.select_all_pacjenty();
""", 
commit=True,
print_out=True)

NOTICE:  W bazie jest juz: 50091 pacjentow!!

no results to fetch
